<a href="https://colab.research.google.com/github/Eezzeldin/candy/blob/chainclassifier/candy_EDA_4_multioutput.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.multioutput import ClassifierChain
from sklearn.metrics import f1_score
from sklearn.metrics import balanced_accuracy_score

In [34]:
#pip list

Package                          Version
-------------------------------- ---------------------
absl-py                          1.4.0
aiohttp                          3.9.3
aiosignal                        1.3.1
alabaster                        0.7.16
albumentations                   1.3.1
altair                           4.2.2
annotated-types                  0.6.0
anyio                            3.7.1
appdirs                          1.4.4
argon2-cffi                      23.1.0
argon2-cffi-bindings             21.2.0
array_record                     0.5.1
arviz                            0.15.1
astropy                          5.3.4
astunparse                       1.6.3
async-timeout                    4.0.3
atpublic                         4.1.0
attrs                            23.2.0
audioread                        3.0.1
autograd                         1.6.2
Babel                            2.14.0
backcall                         0.2.0
beautifulsoup4                   4.12.3


In [39]:
#!pip install --upgrade scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 35.4 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2


In [2]:

# Load the dataset
df = pd.read_csv('/content/candy-data.csv')

# Display the first few rows of the dataset
df.head()

,competitorname,chocolate,fruity,caramel,peanutyalmondy,nougat,crispedricewafer,hard,bar,pluribus,sugarpercent,pricepercent,winpercent
0,100 Grand,1,0,1,0,0,1,0,1,0,0.732,0.860,66.971725
1,3 Musketeers,1,0,0,0,1,0,0,1,0,0.604,0.511,67.602936
2,One dime,0,0,0,0,0,0,0,0,0,0.011,0.116,32.261086
3,One quarter,0,0,0,0,0,0,0,0,0,0.011,0.511,46.116505
4,Air Heads,0,1,0,0,0,0,0,0,0,0.906,0.511,52.341465


In [3]:
df.columns

Index(['competitorname', 'chocolate', 'fruity', 'caramel', 'peanutyalmondy',
       'nougat', 'crispedricewafer', 'hard', 'bar', 'pluribus', 'sugarpercent',
       'pricepercent', 'winpercent'],
      dtype='object')

In [ ]:
'''
Chocolate Group
Bar: 20
Peanutyalmondy: 12
Pluribus: 12
Caramel: 10
Crispedricewafer: 7
Nougat: 6
Hard: 1
Fruity Group
Pluribus: 26
Hard: 12
Caramel: 1
(Other features have zero counts)
Other Group
Pluribus: 6
Caramel: 3
Peanutyalmondy and Bar: 2 each
Hard, Nougat: 2 and 1 respectively
(Crispedricewafer has zero counts)
'''

In [4]:
for c in ['chocolate', 'fruity', 'caramel', 'peanutyalmondy','nougat', 'crispedricewafer', 'hard', 'bar', 'pluribus'] : print (c,df [c].sum())

chocolate 37
fruity 38
caramel 14
peanutyalmondy 14
nougat 7
crispedricewafer 7
hard 15
bar 21
pluribus 44


In [5]:
col_order = ["fruity" , "chocolate" , "pluribus"  , "peanutyalmondy" , "caramel" , "hard" , "crispedricewafer" , "nougat"  ]
df [col_order]

,fruity,chocolate,pluribus,peanutyalmondy,caramel,hard,crispedricewafer,nougat
0,0,1,0,0,1,0,1,0
1,0,1,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
80,1,0,0,0,0,0,0,0
81,1,0,0,0,0,1,0,0
82,1,0,1,0,0,0,0,0
83,0,0,0,0,1,1,0,0


In [19]:
feature_cols = ['sugarpercent','pricepercent','winpercent']
X = df [feature_cols]
X

,sugarpercent,pricepercent,winpercent
0,0.732,0.860,66.971725
1,0.604,0.511,67.602936
2,0.011,0.116,32.261086
3,0.011,0.511,46.116505
4,0.906,0.511,52.341465
...,...,...,...
80,0.220,0.116,45.466282
81,0.093,0.116,39.011898
82,0.313,0.313,44.375519
83,0.186,0.267,41.904308


In [20]:
y = df [col_order]
y

,fruity,chocolate,pluribus,peanutyalmondy,caramel,hard,crispedricewafer,nougat
0,0,1,0,0,1,0,1,0
1,0,1,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
80,1,0,0,0,0,0,0,0
81,1,0,0,0,0,1,0,0
82,1,0,1,0,0,0,0,0
83,0,0,0,0,1,1,0,0


In [19]:
from sklearn.datasets import make_multilabel_classification
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.multioutput import ClassifierChain
X, Y = make_multilabel_classification(
   n_samples=12, n_classes=3, random_state=0
)
X_train, X_test, Y_train, Y_test = train_test_split(
   X, Y, random_state=0
)
base_lr = LogisticRegression(solver='lbfgs', random_state=0)
chain = ClassifierChain(base_lr, order='random', random_state=0)
chain.fit(X_train, Y_train).predict(X_test)
chain.predict_proba(X_test)

array([[0.83873122, 0.94314231, 0.4576149 ],
       [0.88781656, 0.3684047 , 0.26409868],
       [0.03213614, 0.99359669, 0.06257921]])

In [22]:
range (y.shape [1])

range(0, 8)

In [23]:
chain = ClassifierChain(base_lr, order=range (y.shape [1]), random_state=0 )

In [24]:
chain.fit (X,y)

ClassifierChain(base_estimator=LogisticRegression(random_state=0),
                order=range(0, 8), random_state=0)

In [25]:
chain.score (X,y)

0.16470588235294117

In [26]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
X, y = make_classification(n_samples=1000, n_features=4,
                           n_informative=2, n_redundant=0,
                           random_state=0, shuffle=False)
clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(X, y)
print(clf.predict([[0, 0, 0, 0]]))

[1]


In [21]:
base_rf = RandomForestClassifier(max_depth=2, random_state=0)

In [22]:
chain = ClassifierChain(base_rf , order=range (y.shape [1]), random_state=0)

In [23]:
chain.fit (X,y)

ClassifierChain(base_estimator=RandomForestClassifier(max_depth=2,
                                                      random_state=0),
                order=range(0, 8), random_state=0)

In [24]:
chain.score (X,y)

0.2823529411764706

In [25]:
chain.predict (X)

array([[0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0.],
       [1., 0., 1., 0., 0., 0., 0., 0.],
       [1., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0.],
       [1., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0.],
       [1., 0., 1., 0., 0., 0., 0., 0.],
       [1., 0., 1., 0., 0., 0., 0., 0.],
       [1., 0., 1., 0., 0., 0., 0., 0.],
       [1., 0., 1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [1., 0., 1., 0., 0., 0., 0., 0.],
       [1., 0., 1., 0., 0., 0., 0., 0.],
       [1., 0., 1., 0., 0., 0., 0., 0.],
       [1., 0., 1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 1., 

In [29]:
y.iloc [:,0]

0     0
1     0
2     0
3     0
4     1
     ..
80    1
81    1
82    1
83    0
84    0
Name: fruity, Length: 85, dtype: int64

In [31]:
pd.DataFrame ( chain.predict (X) )

,0,1,2,3,4,5,6,7
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
80,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
81,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
82,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
83,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [32]:
f1_score(y.iloc [:,0], pd.DataFrame ( chain.predict (X) ) [0], average='weighted')

0.800500765057727

In [33]:
for i in range (y.shape [1]) :
    print ( i , f1_score(y.iloc [:,i], pd.DataFrame ( chain.predict (X) ) [i], average='weighted') )

0 0.800500765057727
1 0.8588235294117647
2 0.6936928104575163
3 0.8949559184853302
4 0.7603318250377074
5 0.7438330170777989
6 0.8782389029231326
7 0.8782389029231326


In [39]:
for i in range (y.shape [1]) :
    print ( i , balanced_accuracy_score (y.iloc [:,i], pd.DataFrame ( chain.predict (X) ) [i]  , adjusted=True) )

0 0.603023516237402
1 0.7128378378378377
2 0.38580931263858087
3 0.4859154929577465
4 0.0
5 0.0
6 0.0
7 0.0


In [17]:
chain.get_metadata_routing()

{'estimator': {'mapping': [{'callee': 'fit', 'caller': 'fit'}], 'router': {'fit': {'sample_weight': None}, 'score': {'sample_weight': None}}}}